# Format the various LaTeX tables with our results

This notebook has a very simple and limited purpose, to construct correctly formatted tables containing the results that we generated throughout the course of our analyses - this notebook will likely create multiple tables (e.g. global spectral properties and measured positions/rotation angles).

## Import Statements

In [1]:
import pandas as pd
import numpy as np

## Reading the necessary files

This is the sample of clusters which is included in the paper:

In [2]:
samp = pd.read_csv("../../sample_files/X-LoVoCCSI.csv")
samp.head(5)

,LoVoCCSID,Name,start_ra,start_dec,MCXC_Redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec
0,1,A2029,227.734300,5.745471,0.0766,1.3344,227.73000,5.720000,227.734300,5.745471
1,2,A401,44.740000,13.580000,0.0739,1.2421,44.74000,13.580000,NaN,NaN
2,4A,A85North,10.458750,-9.301944,0.0555,1.2103,10.45875,-9.301944,NaN,NaN
3,4B,A85South,10.451487,-9.460007,0.0555,1.2103,10.45875,-9.301944,10.451487,-9.460007
4,5,A3667,303.157313,-56.845978,0.0556,1.1990,303.13000,-56.830000,303.157313,-56.845978


Next file to load includes the centroid coordinates, position angles, and axis ratios for the surface brightness distribution of the X-ray emitting ICM:

In [3]:
cent = pd.read_csv("../../outputs/coordinates/xmm_500kpc_centroid.csv")
cent.head(5)

,LoVoCCS_name,cent_ra,cent_ra-,cent_ra+,cent_dec,cent_dec-,cent_dec+,position_angle,position_angle-,position_angle+,ax_ratio,ax_ratio-,ax_ratio+
0,LoVoCCS-1,227.73319,0.00002,0.00002,5.74323,0.00002,0.00002,-163.2287,0.1878,0.2336,0.9179,0.0010,0.0012
1,LoVoCCS-2,44.73666,0.00015,0.00020,13.57764,0.00016,0.00016,-66.6192,4.4151,4.5963,0.9725,0.0055,0.0046
2,LoVoCCS-4,10.46009,0.00003,0.00003,-9.30464,0.00005,0.00003,157.3862,0.1785,0.1713,0.8917,0.0011,0.0009
3,LoVoCCS-5,303.14178,0.00013,0.00013,-56.83836,0.00007,0.00007,-49.9254,0.2451,0.1678,0.8331,0.0013,0.0011
4,LoVoCCS-7,330.47797,0.00019,0.00020,-59.94793,0.00011,0.00012,-168.0726,0.8656,0.8021,0.9492,0.0036,0.0038


In the same vein, we load the file containing the coordinates of the X-ray peak of each cluster:

In [4]:
peak = pd.read_csv("../../outputs/coordinates/xmm_peak_coords.csv")
peak.head(5)

,LoVoCCS_name,peak_ra,peak_dec
0,LoVoCCS-1,227.733253,5.745184
1,LoVoCCS-2,44.750675,13.594567
2,LoVoCCS-4,10.460369,-9.303375
3,LoVoCCS-5,303.100675,-56.851435
4,LoVoCCS-7,330.466217,-59.948751


## Generating the measured position table string

Here we generate the string of LaTeX code that can be copied in as the data body of the table:

In [17]:
round_pos = 3
with_err = r" ${v}_{{-{me}}}^{{+{pe}}}$ &"

In [26]:
lines = ""
for row_ind, row in samp.iterrows():
    rel_name = row['Name']
    if rel_name == "RX J0820.9+0751/RXC J0821.0+0751":
        rel_name = "RXC J0821.0+0751"
    
    rel_lov_id = row['LoVoCCSID']
    rel_lov_name = 'LoVoCCS-' + rel_lov_id
    rel_cent = cent[cent['LoVoCCS_name'] == rel_lov_name]
    rel_peak = peak[peak['LoVoCCS_name'] == rel_lov_name]
    
    if len(rel_cent) == 0:
        continue
    else:
        rel_cent = rel_cent.iloc[0]
        rel_peak = rel_peak.iloc[0]
    
    res_line = "{i} & ".format(i=rel_lov_id)
    
    c_ra = rel_cent[['cent_ra', 'cent_ra-', 'cent_ra+']].values.astype(float).round(round_pos)
    res_line += with_err.format(v=c_ra[0], me=c_ra[1], pe=c_ra[2])
    c_dec = rel_cent[['cent_dec', 'cent_dec-', 'cent_dec+']].values.astype(float).round(round_pos)
    res_line += with_err.format(v=c_dec[0], me=c_dec[1], pe=c_dec[2])
    
    c_pa = rel_cent[['position_angle', 'position_angle-', 'position_angle+']].values.astype(float).round(2)
    res_line += with_err.format(v=c_pa[0], me=c_pa[1], pe=c_pa[2])
    
    c_ax = rel_cent[['ax_ratio', 'ax_ratio-', 'ax_ratio+']].values.astype(float).round(3)
    res_line += with_err.format(v=c_ax[0], me=c_ax[1], pe=c_ax[2])
    
    
    res_line += "{r} & {d}".format(r=rel_peak['peak_ra'].round(round_pos), 
                                   d=rel_peak['peak_dec'].round(round_pos))
    
    
    res_line += '\\\ \n'
    lines += res_line
print(lines)

1 &  $227.733_{-0.0}^{+0.0}$ & $5.743_{-0.0}^{+0.0}$ & $-163.23_{-0.19}^{+0.23}$ & $0.918_{-0.001}^{+0.001}$ &227.733 & 5.745\\ 
2 &  $44.737_{-0.0}^{+0.0}$ & $13.578_{-0.0}^{+0.0}$ & $-66.62_{-4.42}^{+4.6}$ & $0.972_{-0.006}^{+0.005}$ &44.751 & 13.595\\ 
5 &  $303.142_{-0.0}^{+0.0}$ & $-56.838_{-0.0}^{+0.0}$ & $-49.93_{-0.25}^{+0.17}$ & $0.833_{-0.001}^{+0.001}$ &303.101 & -56.851\\ 
7 &  $330.478_{-0.0}^{+0.0}$ & $-59.948_{-0.0}^{+0.0}$ & $-168.07_{-0.87}^{+0.8}$ & $0.949_{-0.004}^{+0.004}$ &330.466 & -59.949\\ 
9 &  $67.844_{-0.0}^{+0.0}$ & $-61.432_{-0.0}^{+0.0}$ & $237.58_{-0.45}^{+0.59}$ & $0.916_{-0.002}^{+0.002}$ &67.812 & -61.454\\ 
10 &  $194.842_{-0.0}^{+0.0}$ & $-4.193_{-0.0}^{+0.0}$ & $250.55_{-2.94}^{+2.98}$ & $0.956_{-0.006}^{+0.007}$ &194.836 & -4.194\\ 
11 &  $137.29_{-0.0}^{+0.0}$ & $-9.673_{-0.0}^{+0.0}$ & $-85.27_{-0.38}^{+0.28}$ & $0.97_{-0.002}^{+0.002}$ &137.325 & -9.685\\ 
12 &  $206.866_{-0.0}^{+0.0}$ & $-32.852_{-0.0}^{+0.0}$ & $-171.59_{-0.38}^{+0.36}$ & $0.9

## Generating the global spectral properties table string